In [3]:
import os
import webbrowser
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import plotly.offline as pyo
from plotly.subplots import make_subplots
from datetime import datetime
import warnings
import sqlite3

warnings.filterwarnings('ignore')

# Register Edge browser
EDGE_PATH = r"C:/Program Files (x86)/Microsoft/Edge/Application/msedge.exe" # Ensure this path is correct for your system
# Check if Edge path exists, if not, don't register and let webbrowser use default
if os.path.exists(EDGE_PATH):
    webbrowser.register('edge', None, webbrowser.BackgroundBrowser(EDGE_PATH))
    BROWSER_TO_USE = 'edge'
else:
    print(f"Edge browser not found at {EDGE_PATH}. Using default system browser.")
    BROWSER_TO_USE = None # This will make webbrowser.get() use the default browser

# --- MODIFIED COLOR PALETTE ---
COLOR_PALETTE = {
    'background': '#000000',      # Black background from image
    'text': '#00FF41',            # Bright green text/icons from image
    'primary': '#00FF41',         # Using green as the primary color for charts/titles
    'success': "#00FF41",         # Green for winning trades/positive P&L
    'danger': "#FF0000",          # A vibrant red for losses, contrasting with the green
    'warning': '#FFFF00',         # A bright yellow for neutral highlights (e.g., capital line)
    'purple': '#00FF41',          # Using green for thematic consistency
    'gold': '#00FF41',            # Using green for thematic consistency
    'pink': '#00FF41',            # Using green for thematic consistency
    'gradient_start': '#00FF41',
    'gradient_end': '#00FF41',
    'surface': '#00FF41'          # Using green for table borders and accents
}

def format_indian_number(x, prefix="₹"):
    try:
        num = float(x)
    except:
        return str(x) if pd.notna(x) else "N/A" # Handle NaN
    integer_part, decimal_part = f"{num:.2f}".split('.')
    rev = integer_part[::-1]
    groups = [rev[:3]]
    rev = rev[3:]
    while rev:
        groups.append(rev[:2])
        rev = rev[2:]
    indian_int = ','.join(groups)[::-1]
    return f"{prefix}{indian_int}.{decimal_part}"

class TradingJournal:
    def __init__(self, starting_capital=1_400_000, lot_size=75, cost_per_lot=80, current_lot_size=15):
        self.starting_capital = starting_capital
        self.lot_size = lot_size # This is the historical lot size for cost calculation
        self.cost_per_lot = cost_per_lot
        self.current_lot_size = current_lot_size # The current lot size for new metrics
        self.data = None
        self.metrics = {}

    def load_data(self, trades_data):
        df = pd.DataFrame(trades_data)
        df['Entry Date'] = pd.to_datetime(df['Entry Date'], dayfirst=True, errors='coerce')
        df['Exit Date'] = pd.to_datetime(df['Exit Date'], dayfirst=True, errors='coerce')
        
        df.dropna(subset=['Entry Date', 'Exit Date'], inplace=True)
        if df.empty:
            print("Warning: No valid date entries found in trade data after date parsing.")
            self.data = pd.DataFrame()
            return self.data

        df = df.sort_values('Exit Date').reset_index(drop=True)

        numeric_cols = ['Quantity', 'Profit']
        for col in numeric_cols:
            df[col] = pd.to_numeric(df[col], errors='coerce')
        
        df.dropna(subset=numeric_cols, inplace=True)
        if df.empty:
            print("Warning: No valid numeric entries for Quantity/Profit after parsing.")
            self.data = pd.DataFrame()
            return self.data
            
        df['Lots'] = (df['Quantity'] / self.lot_size).round().astype(int)
        df['Trading Costs'] = df['Lots'] * self.cost_per_lot
        df['Net Profit'] = df['Profit'] - df['Trading Costs']

        df['Cumulative P&L'] = df['Net Profit'].cumsum()
        df['Equity'] = self.starting_capital + df['Cumulative P&L']
        df['Peak'] = df['Equity'].cummax()
        df['Drawdown'] = df['Equity'] - df['Peak']
        df['Drawdown %'] = np.where(df['Peak'] != 0, (df['Drawdown'] / df['Peak']) * 100, 0)
        
        self.data = df
        return df

    # <<< CHANGE START >>>
    # The logic inside this function has been updated to reflect the new requests.
    def calculate_metrics(self):
        if self.data is None or self.data.empty:
            print("No data loaded to calculate metrics.")
            # Update metric keys for the "no data" scenario
            metric_keys = [
                'Starting Capital', 'Start Date', 'End Date', 'Trading Days',
                'Total Trades', 'Winning Trades', 'Losing Trades', 'Winning Rate',
                'Avg Win', 'Avg Loss', 'Win/Loss Ratio', 'Total Net P&L', 'Total Net P&L %',
                'Max Drawdown', 'Max Drawdown %', 'Max Win Streak', 'Max Loss Streak',
                'Max DD Duration (days)', 'Current DD Duration (days)', 
                'Historical Lot Size', 'Cost per Lot',
                'Highest Cumulative Profit', 'Current Capital', 'Highest Capital Value',
                'Current Lot Size', 'Capital Allocation per Lot'
            ]
            self.metrics = {key: "N/A" for key in metric_keys}
            self.metrics['Starting Capital'] = format_indian_number(self.starting_capital)
            self.metrics['Historical Lot Size'] = str(self.lot_size)
            self.metrics['Cost per Lot'] = format_indian_number(self.cost_per_lot)
            self.metrics['Current Lot Size'] = str(self.current_lot_size)
            return self.metrics

        df = self.data
        winners = df[df['Net Profit'] > 0]['Net Profit']
        losers = df[df['Net Profit'] < 0]['Net Profit']

        start_date = df['Entry Date'].min()
        end_date = df['Exit Date'].max()
        trading_days = (end_date - start_date).days if pd.notna(start_date) and pd.notna(end_date) else 0
        total_trades = len(df)
        
        if total_trades == 0:
            return self.calculate_metrics()

        total_net_profit = df['Net Profit'].sum()
        pct_net_profit = (total_net_profit / self.starting_capital) * 100 if self.starting_capital != 0 else 0

        df['Win'] = df['Net Profit'] > 0
        df['Streak'] = (df['Win'] != df['Win'].shift()).cumsum()
        streaks = df.groupby('Streak')['Win'].agg(['first','count'])
        win_streaks = streaks[streaks['first']==True]['count']
        loss_streaks = streaks[streaks['first']==False]['count']
        max_win = int(win_streaks.max()) if not win_streaks.empty else 0
        max_loss = int(loss_streaks.max()) if not loss_streaks.empty else 0

        max_dd = df['Drawdown'].min() if not df['Drawdown'].empty else 0
        max_dd_pct = df['Drawdown %'].min() if not df['Drawdown %'].empty else 0
        periods = self._calculate_drawdown_periods()
        max_dd_days = max((p['days'] for p in periods), default=0)
        current_dd_days = periods[-1]['days'] if periods and periods[-1]['ongoing'] else 0

        # Corrected and new calculations as per your request
        highest_cumulative_profit = df['Cumulative P&L'].max() if not df['Cumulative P&L'].empty else 0
        current_capital = df['Equity'].iloc[-1] if not df['Equity'].empty else self.starting_capital
        highest_capital = df['Peak'].max() if not df['Peak'].empty else self.starting_capital
        
        capital_per_lot = 0
        if self.current_lot_size > 0:
            raw_alloc = highest_capital / self.current_lot_size
            capital_per_lot = round(raw_alloc / 100000) * 100000 # Round to nearest lakh

        self.metrics = {
            'Starting Capital': format_indian_number(self.starting_capital),
            'Start Date': start_date.strftime('%Y-%m-%d') if pd.notna(start_date) else "N/A",
            'End Date': end_date.strftime('%Y-%m-%d') if pd.notna(end_date) else "N/A",
            'Trading Days': str(trading_days),
            'Total Trades': str(total_trades),
            'Winning Trades': str(len(winners)),
            'Losing Trades': str(len(losers)),
            'Winning Rate': f"{len(winners)/total_trades*100:.2f}%" if total_trades > 0 else "0.00%",
            'Avg Win': format_indian_number(winners.mean()) if not winners.empty else format_indian_number(0),
            'Avg Loss': format_indian_number(losers.mean()) if not losers.empty else format_indian_number(0),
            'Win/Loss Ratio': f"{abs(winners.mean()/losers.mean()):.2f}" if not winners.empty and not losers.empty and losers.mean() != 0 else "N/A",
            'Total Net P&L': format_indian_number(total_net_profit),
            'Total Net P&L %': f"{pct_net_profit:.2f}%",
            'Max Drawdown': format_indian_number(max_dd),
            'Max Drawdown %': f"{max_dd_pct:.2f}%",
            'Max Win Streak': str(max_win),
            'Max Loss Streak': str(max_loss),
            'Max DD Duration (days)': str(max_dd_days),
            'Current DD Duration (days)': str(current_dd_days),
            'Historical Lot Size': str(self.lot_size),
            'Cost per Lot': format_indian_number(self.cost_per_lot),
            'Highest Cumulative Profit': format_indian_number(highest_cumulative_profit),
            'Current Capital': format_indian_number(current_capital),
            'Highest Capital Value': format_indian_number(highest_capital),
            'Current Lot Size': str(self.current_lot_size),
            'Capital Allocation per Lot': format_indian_number(capital_per_lot)
        }
        return self.metrics
    # <<< CHANGE END >>>

    def _calculate_drawdown_periods(self):
        if self.data is None or self.data.empty:
            return []
        periods, in_dd, start_dd_date = [], False, None
        for _, r in self.data.iterrows():
            if r['Drawdown'] < 0 and not in_dd:
                in_dd, start_dd_date = True, r['Exit Date']
            elif r['Drawdown'] >= 0 and in_dd:
                if pd.notna(start_dd_date) and pd.notna(r['Exit Date']):
                    days = (r['Exit Date'] - start_dd_date).days
                    periods.append({'start': start_dd_date, 'end': r['Exit Date'], 'days': days, 'ongoing': False})
                in_dd = False
        if in_dd and pd.notna(start_dd_date):
            last_exit_date = self.data['Exit Date'].iloc[-1]
            if pd.notna(last_exit_date):
                days = (last_exit_date - start_dd_date).days
                periods.append({'start': start_dd_date, 'end': last_exit_date, 'days': days, 'ongoing': True})
        return periods

    def _save_html(self, path, html_content):
        with open(path,'w',encoding='utf-8') as f:
            f.write(f"<html><head><meta charset='utf-8'></head><body style='background:{COLOR_PALETTE['background']};color:{COLOR_PALETTE['text']};font-family:\"Consolas\", \"Menlo\", \"Monaco\", \"Courier New\", monospace;margin:0;padding:20px;'>{html_content}</body></html>")
        return path

    def save_metrics_page(self, out_dir):
        rows = ""
        for k,v_val in self.metrics.items():
            raw_val_str = str(v_val).replace('₹','').replace('%','').replace(',','') if isinstance(v_val,str) else str(v_val)
            color = COLOR_PALETTE['text'] 
            try:
                num_val = float(raw_val_str)
                neutral_keys = [
                    'Historical Lot Size', 'Cost per Lot', 'Trading Days', 
                    'Total Trades', 'Max Win Streak', 'Max Loss Streak', 
                    'Max DD Duration (days)', 'Current DD Duration (days)',
                    'Starting Capital', 'Current Lot Size'
                ]
                if k in ['Max Drawdown %', 'Max Drawdown']: 
                    color = COLOR_PALETTE['danger'] if num_val < 0 else COLOR_PALETTE['text']
                elif k not in neutral_keys:
                    if num_val > 0 : color = COLOR_PALETTE['success']
                    elif num_val < 0: color = COLOR_PALETTE['danger']
            except ValueError:
                pass 
            
            rows += (f"<tr><th style='padding:12px;text-align:left;color:{COLOR_PALETTE['text']};border-bottom: 1px solid {COLOR_PALETTE['surface']};'>{k}</th>"
                     f"<td style='padding:12px;text-align:right;color:{color};border-bottom: 1px solid {COLOR_PALETTE['surface']};'>{v_val}</td></tr>")
        html_content=f"<h1 style='text-align:center;color:{COLOR_PALETTE['primary']};'>📈 Performance Metrics</h1><table style='width:80%;margin-left:auto;margin-right:auto;border-collapse:collapse;box-shadow: 0 4px 8px 0 {COLOR_PALETTE['surface']};border-radius:8px;overflow:hidden;'>{rows}</table>"
        path=os.path.join(out_dir,'metrics.html')
        return self._save_html(path,html_content)

    def save_trade_table_page(self, out_dir):
        if self.data is None or self.data.empty:
            html_content = f"<h1 style='text-align:center;color:{COLOR_PALETTE['primary']};'>📊 Daily Trading Summary</h1><p style='text-align:center;'>No trade data available.</p>"
            path = os.path.join(out_dir, 'trade_summary.html')
            return self._save_html(path, html_content)

        df_copy = self.data.copy()
        df_copy['ExDate'] = df_copy['Exit Date'].dt.date

        agg_df = df_copy.groupby(['Symbol', 'ExDate']).agg(
            {'Entry Date': 'first', 'Exit Date': 'first', 
             'Quantity': 'sum', 'Lots': 'sum', 
             'Net Profit': 'sum', 'Trading Costs': 'sum'}
        ).reset_index()
        
        df2 = agg_df.sort_values('Exit Date')
        df2['Net P/L per Lot'] = np.where(df2['Lots'] != 0, (df2['Net Profit'] / df2['Lots']).round(2), 0)
        
        df2['Month'] = df2['Exit Date'].dt.to_period('M')
        monthly_sum = df2.groupby('Month')['Net Profit'].sum().to_dict()
        df2['Monthly Net P&L'] = df2['Month'].map(monthly_sum)

        df2['Cumulative Net P&L'] = df2['Net Profit'].cumsum()
        df2['Cumulative P&L %'] = (df2['Cumulative Net P&L'] / self.starting_capital * 100).round(2)
        
        df2['Cumulative Capital'] = self.starting_capital + df2['Cumulative Net P&L']
        df2['Peak Capital'] = df2['Cumulative Capital'].cummax()
        df2['DD %'] = np.where(df2['Peak Capital'] != 0, ((df2['Cumulative Capital'] - df2['Peak Capital']) / df2['Peak Capital'] * 100).round(2), 0)

        header_row = f"""<tr style="border: 2px solid {COLOR_PALETTE['primary']}; background-color: {COLOR_PALETTE['background']};">
                        <th>Month</th><th>Symbol</th><th>Entry</th><th>Exit</th>
                        <th>Total Qty</th><th>Total Lots</th><th>Costs</th><th>Net P/L p. Lot</th>
                        <th>Net P/L</th><th>Cum. Net P/L</th><th>Cum. P/L %</th><th>Monthly Net P&L</th>
                        <th>Cum. Capital</th><th>DD %</th>
                      </tr>"""
        
        body_rows = ""
        for _, r_row in df2.iterrows():
            pnl_color = COLOR_PALETTE['success'] if r_row['Net Profit'] >= 0 else COLOR_PALETTE['danger']
            dd_color = COLOR_PALETTE['danger'] if r_row['DD %'] < 0 else COLOR_PALETTE['text']
            cum_pnl_pct_color = COLOR_PALETTE['success'] if r_row['Cumulative P&L %'] >= 0 else COLOR_PALETTE['danger']
            
            body_rows += (f"<tr style='border-bottom: 1px solid {COLOR_PALETTE['surface']};'>"
                          f"<td style='padding:8px;'>{r_row['Month']}</td><td style='padding:8px;'>{r_row['Symbol']}</td>"
                          f"<td style='padding:8px;'>{r_row['Entry Date'].date()}</td><td style='padding:8px;'>{r_row['Exit Date'].date()}</td>"
                          f"<td style='padding:8px;text-align:right;'>{r_row['Quantity']}</td><td style='padding:8px;text-align:right;'>{r_row['Lots']}</td>"
                          f"<td style='padding:8px;text-align:right;'>{format_indian_number(r_row['Trading Costs'], prefix='')}</td>"
                          f"<td style='padding:8px;text-align:right;'>{r_row['Net P/L per Lot']:.2f}</td>"
                          f"<td style='padding:8px;text-align:right;color:{pnl_color};'>{format_indian_number(r_row['Net Profit'], prefix='')}</td>"
                          f"<td style='padding:8px;text-align:right;'>{format_indian_number(r_row['Cumulative Net P&L'], prefix='')}</td>"
                          f"<td style='padding:8px;text-align:right;color:{cum_pnl_pct_color};'>{r_row['Cumulative P&L %']:.2f}%</td>"
                          f"<td style='padding:8px;text-align:right;color:{COLOR_PALETTE['success'] if r_row['Monthly Net P&L'] >= 0 else COLOR_PALETTE['danger']};'>{format_indian_number(r_row['Monthly Net P&L'], prefix='')}</td>"
                          f"<td style='padding:8px;text-align:right;'>{format_indian_number(r_row['Cumulative Capital'], prefix='')}</td>"
                          f"<td style='padding:8px;text-align:right;color:{dd_color};'>{r_row['DD %']}%</td></tr>")
            
        html_content = f"<h1 style='text-align:center;color:{COLOR_PALETTE['primary']};'>📊 Daily Trading Summary</h1><table style='width:100%;border-collapse:collapse;text-align:left;'>{header_row}{body_rows}</table>"
        path = os.path.join(out_dir, 'trade_summary.html')
        return self._save_html(path, html_content)

    def _create_empty_figure(self, title="No Data Available"):
        fig = go.Figure()
        fig.update_layout(
            title_text=title,
            title_x=0.5,
            plot_bgcolor=COLOR_PALETTE['background'],
            paper_bgcolor=COLOR_PALETTE['background'],
            font_color=COLOR_PALETTE['text'],
            xaxis={'visible': False},
            yaxis={'visible': False},
            annotations=[{
                'text': "No trade data to display for this chart.",
                'xref': "paper", 'yref': "paper",
                'showarrow': False, 'font': {'size': 16, 'color': COLOR_PALETTE['text']}
            }]
        )
        return fig

    def create_equity_curve_chart(self):
        if self.data is None or self.data.empty: return self._create_empty_figure("Equity Curve (Net of Costs)")
        fig=go.Figure()
        fig.add_trace(go.Scatter(x=self.data['Exit Date'],y=self.data['Equity'],mode='lines',name='Equity',line=dict(color=COLOR_PALETTE['primary'],width=3)))
        fig.add_hline(y=self.starting_capital,line_dash='dash',line_color=COLOR_PALETTE['warning'],annotation_text='Starting Capital',annotation_position='bottom right')
        fig.update_layout(title_text='Equity Curve (Net of Costs)', title_x=0.5, plot_bgcolor=COLOR_PALETTE['background'],paper_bgcolor=COLOR_PALETTE['background'],font_color=COLOR_PALETTE['text'], yaxis_title="Equity")
        return fig

    def create_drawdown_chart(self):
        if self.data is None or self.data.empty: return self._create_empty_figure("Drawdown Percentage of Peak Equity")
        fig=go.Figure()
        fig.add_trace(go.Scatter(x=self.data['Exit Date'],y=self.data['Drawdown %'],mode='lines',name='Drawdown %',fill='tozeroy',line_color=COLOR_PALETTE['danger'],fillcolor='rgba(255, 0, 0, 0.3)'))
        fig.update_layout(title_text='Drawdown Percentage of Peak Equity', title_x=0.5, plot_bgcolor=COLOR_PALETTE['background'],paper_bgcolor=COLOR_PALETTE['background'],font_color=COLOR_PALETTE['text'], yaxis_title="Drawdown %", yaxis_ticksuffix='%')
        return fig

    def create_pnl_distribution_chart(self):
        if self.data is None or self.data.empty: return self._create_empty_figure("Net P&L Distribution (Post Costs)")
        net_profit_series = self.data['Net Profit']
        fig=go.Figure()
        fig.add_trace(go.Histogram(x=net_profit_series,nbinsx=30,name='Net P&L',marker=dict(color=COLOR_PALETTE['primary'],line=dict(color=COLOR_PALETTE['background'],width=1))))
        fig.update_layout(title_text='Net P&L Distribution (Post Costs)', title_x=0.5, plot_bgcolor=COLOR_PALETTE['background'],paper_bgcolor=COLOR_PALETTE['background'],font_color=COLOR_PALETTE['text'], xaxis_title="Net P&L")
        return fig

    def create_monthly_pnl_chart(self):
        if self.data is None or self.data.empty: return self._create_empty_figure("Monthly Net P&L (Post Costs)")
        monthly_net_pnl = self.data.set_index('Exit Date')['Net Profit'].resample('M').sum()
        if monthly_net_pnl.empty: return self._create_empty_figure("Monthly Net P&L (Post Costs) - No Monthly Data")
        colors=[COLOR_PALETTE['success'] if v_val>=0 else COLOR_PALETTE['danger'] for v_val in monthly_net_pnl]
        fig=go.Figure([go.Bar(x=monthly_net_pnl.index.strftime('%Y-%b'),y=monthly_net_pnl.values,name='Monthly Net P&L',marker_color=colors)])
        fig.update_layout(title_text='Monthly Net P&L (Post Costs)', title_x=0.5, plot_bgcolor=COLOR_PALETTE['background'],paper_bgcolor=COLOR_PALETTE['background'],font_color=COLOR_PALETTE['text'], yaxis_title="Net P&L")
        return fig

    def create_winloss_pie_chart(self):
        if self.data is None or self.data.empty: return self._create_empty_figure("Trade Outcome Ratio (Based on Net P&L)")
        wins = len(self.data[self.data['Net Profit'] > 0])
        losses = len(self.data[self.data['Net Profit'] < 0])
        breakeven = len(self.data[self.data['Net Profit'] == 0])
        
        labels = ['Winning Trades', 'Losing Trades']
        values = [wins, losses]
        chart_colors = [COLOR_PALETTE['success'], COLOR_PALETTE['danger']]

        if breakeven > 0:
            labels.append('Breakeven Trades')
            values.append(breakeven)
            chart_colors.append(COLOR_PALETTE['warning'])
        
        if not values or sum(values) == 0:
             return self._create_empty_figure("Trade Outcome Ratio - No Trades with P&L")

        fig=go.Figure(go.Pie(labels=labels, values=values, marker_colors=chart_colors, hole=0.3))
        fig.update_layout(title_text='Trade Outcome Ratio (Based on Net P&L)', title_x=0.5, plot_bgcolor=COLOR_PALETTE['background'],paper_bgcolor=COLOR_PALETTE['background'],font_color=COLOR_PALETTE['text'])
        return fig

    def create_cumulative_pnl_chart(self):
        if self.data is None or self.data.empty: return self._create_empty_figure("Cumulative Net P&L (Post Costs)")
        fig = go.Figure()
        fig.add_trace(go.Scatter(x=self.data['Exit Date'], y=self.data['Cumulative P&L'], name='Cumulative Net P&L', mode='lines', line=dict(color=COLOR_PALETTE['primary'], width=3)))
        fig.update_layout(title_text='Cumulative Net P&L (Post Costs)', title_x=0.5, plot_bgcolor=COLOR_PALETTE['background'], paper_bgcolor=COLOR_PALETTE['background'], font_color=COLOR_PALETTE['text'], yaxis_title="Cumulative Net P&L")
        return fig

    def create_combined_dashboard(self):
        if self.data is None or self.data.empty:
            return self._create_empty_figure("Comprehensive Trading Dashboard - No Data Available")

        fig=make_subplots(rows=3,cols=2,
                          subplot_titles=['Equity Curve (Net)','Drawdown % of Peak Equity','Net P&L Distribution','Monthly Net P&L','Trade Outcome Ratio (Net)','Cumulative Net P&L'],
                          specs=[[{},{}],
                                 [{},{}],
                                 [{'type':'domain'},{}]])
        
        eq_fig = self.create_equity_curve_chart()
        if eq_fig.data: fig.add_trace(eq_fig.data[0],row=1,col=1)
        if hasattr(eq_fig.layout, 'shapes') and eq_fig.layout.shapes:
             for shape in eq_fig.layout.shapes: fig.add_shape(shape, row=1, col=1)

        dd_fig = self.create_drawdown_chart()
        if dd_fig.data: fig.add_trace(dd_fig.data[0],row=1,col=2)

        dist_fig = self.create_pnl_distribution_chart()
        if dist_fig.data: fig.add_trace(dist_fig.data[0],row=2,col=1)
        
        month_fig = self.create_monthly_pnl_chart()
        if month_fig.data: fig.add_trace(month_fig.data[0],row=2,col=2)
        
        pie_fig = self.create_winloss_pie_chart()
        if pie_fig.data: fig.add_trace(pie_fig.data[0],row=3,col=1)
        
        cum_pnl_fig = self.create_cumulative_pnl_chart()
        if cum_pnl_fig.data: fig.add_trace(cum_pnl_fig.data[0],row=3,col=2)
        
        fig.update_layout(height=1300, showlegend=False, title_text="Comprehensive Trading Dashboard (Net of Costs)", title_x=0.5,
                          plot_bgcolor=COLOR_PALETTE['background'],paper_bgcolor=COLOR_PALETTE['background'],font_color=COLOR_PALETTE['text'])
        
        fig.update_yaxes(title_text="Equity", row=1, col=1)
        fig.update_yaxes(title_text="Drawdown %", ticksuffix="%", row=1, col=2)
        fig.update_xaxes(title_text="Net P&L", row=2, col=1) 
        fig.update_yaxes(title_text="Frequency", row=2, col=1) 
        fig.update_yaxes(title_text="Net P&L", row=2, col=2)
        fig.update_yaxes(title_text="Cumulative Net P&L", row=3, col=2)

        return fig

    def save_plotly_charts(self, out_dir):
        chart_specs=[
            ('equity_net',self.create_equity_curve_chart()),
            ('drawdown_pct',self.create_drawdown_chart()),
            ('pnl_dist_net',self.create_pnl_distribution_chart()),
            ('monthly_pnl_net',self.create_monthly_pnl_chart()),
            ('winloss_pie_net',self.create_winloss_pie_chart()),
            ('cumulative_net_pnl', self.create_cumulative_pnl_chart()),
            ('dashboard_net',self.create_combined_dashboard())
        ]
        paths=[]
        for name, fig_obj in chart_specs:
            path_to_save = os.path.join(out_dir,f"{name}.html")
            if fig_obj.data: 
                pyo.plot(fig_obj,filename=path_to_save,auto_open=False)
                paths.append(path_to_save)
            elif hasattr(fig_obj.layout, 'annotations') and fig_obj.layout.annotations and "No trade data" in str(fig_obj.layout.annotations[0].text):
                pyo.plot(fig_obj,filename=path_to_save,auto_open=False)
                paths.append(path_to_save)
            else:
                print(f"Skipping saving chart (figure appears empty or invalid): {name}.html")
        return paths

    def generate_and_open_reports(self, out_dir="reports"):
        os.makedirs(out_dir,exist_ok=True)
        
        try:
            browser_obj = webbrowser.get(BROWSER_TO_USE) if BROWSER_TO_USE else webbrowser.get()
        except webbrowser.Error:
            print(f"Could not get browser '{BROWSER_TO_USE}'. Using default system browser to open reports.")
            browser_obj = webbrowser.get() 

        metrics_page_path = self.save_metrics_page(out_dir)
        trade_table_page_path = self.save_trade_table_page(out_dir)
        all_chart_paths = self.save_plotly_charts(out_dir) 
        
        if metrics_page_path and os.path.exists(metrics_page_path):
            browser_obj.open_new_tab(f"file:///{os.path.abspath(metrics_page_path)}")
        
        if trade_table_page_path and os.path.exists(trade_table_page_path):
            browser_obj.open_new_tab(f"file:///{os.path.abspath(trade_table_page_path)}")

        dashboard_filename = "dashboard_net.html"
        dashboard_path = os.path.join(out_dir, dashboard_filename)
        if any(os.path.normpath(dashboard_path) == os.path.normpath(p) for p in all_chart_paths) and os.path.exists(dashboard_path):
            browser_obj.open_new_tab(f"file:///{os.path.abspath(dashboard_path)}")

        for p_chart in all_chart_paths:
            if p_chart and os.path.exists(p_chart) and os.path.basename(p_chart) != dashboard_filename:
                browser_obj.open_new_tab(f"file:///{os.path.abspath(p_chart)}")
        
        print(f"\nReports and charts generated in '{os.path.abspath(out_dir)}' directory.")
        print("If reports did not open automatically, please check your browser settings or open them manually from the directory.")


def load_sqlite_data(db_filepath, table_name="fo_consolidated_data"):
    try:
        conn = sqlite3.connect(db_filepath)
        cursor = conn.cursor()
        cursor.execute(f"SELECT name FROM sqlite_master WHERE type='table' AND name='{table_name}';")
        if cursor.fetchone() is None:
            print(f"Error: Table '{table_name}' does not exist in the database '{db_filepath}'.")
            conn.close()
            return None
        
        query = f"SELECT * FROM {table_name}"
        df = pd.read_sql_query(query, conn)
        conn.close()

        df.columns = df.columns.str.strip().str.replace(' ', '_')

        if 'Cleaned_Symbol' in df.columns:
            df = df.rename(columns={'Cleaned_Symbol': 'Symbol'})
        elif 'Symbol' not in df.columns:
            symbol_col_found = False
            for col_name in df.columns:
                if col_name.lower() == 'symbol':
                    df = df.rename(columns={col_name: 'Symbol'})
                    symbol_col_found = True
                    break
            if not symbol_col_found:
                print("Error: No 'Symbol' or 'Cleaned_Symbol' column found in the database table.")
                return None

        rename_map = {
            'entry_date': 'Entry Date', 'exit_date': 'Exit Date',
            'buy_value': 'Buy Value', 'sell_value': 'Sell Value',
            'profit_/_loss': 'Profit', 'profit': 'Profit',
            'turn_over': 'Turnover', 'turnover': 'Turnover'
        }
        
        current_columns = list(df.columns)
        for col_name_original in current_columns:
            col_name_lower = col_name_original.lower().replace(' ', '_')
            if col_name_lower in rename_map and rename_map[col_name_lower] not in df.columns:
                df.rename(columns={col_name_original: rename_map[col_name_lower]}, inplace=True)

        req_cols = ['Symbol', 'Entry Date', 'Exit Date', 'Quantity', 'Buy Value', 'Sell Value', 'Profit', 'Turnover']
        
        missing_cols = [c for c in req_cols if c not in df.columns]
        if missing_cols:
            print(f"Error: Missing critical columns after attempting to standardize names in SQLite table '{table_name}': {missing_cols}")
            print(f"Available columns after standardization: {list(df.columns)}")
            return None
            
        for col in ['Quantity', 'Profit', 'Buy Value', 'Sell Value', 'Turnover']:
            df[col] = pd.to_numeric(df[col], errors='coerce')
        
        df.dropna(subset=['Quantity', 'Profit', 'Entry Date', 'Exit Date'], inplace=True)

        return df[req_cols].to_dict('records')

    except sqlite3.Error as e:
        print(f"SQLite error: {e}")
        return None
    except Exception as e:
        print(f"An unexpected error occurred during SQLite data loading: {e}")
        import traceback
        traceback.print_exc()
        return None

if __name__=="__main__":
    db_path = r"D:\\TBPL.db" # <--- IMPORTANT: UPDATE THIS PATH
    table_name = "fo_consolidated_data" 

    COST_PER_LOT = 10 
    HISTORICAL_LOT_SIZE = 75 # The lot size used for calculating costs on historical data
    CURRENT_LOT_SIZE = 6     # The current lot size for new capital allocation metrics
    STARTING_CAPITAL = 1_400_000
    REPORTS_OUTPUT_DIRECTORY = "trading_reports_net_final"

    if not os.path.exists(db_path):
        print(f"Error: Database file not found at '{db_path}'. Please check the path and update it in the script.")
        exit(1)

    trades_list = load_sqlite_data(db_path, table_name)

    if not trades_list:
        print("Critical Error: Could not load or process any trade data from the SQLite database. Please check logs for details (e.g., missing columns, incorrect table name, or DB file issues). Exiting.")
        exit(1)
    
    print(f"Successfully loaded {len(trades_list)} raw trade records from '{db_path}'.")

    journal = TradingJournal(
        starting_capital=STARTING_CAPITAL,
        lot_size=HISTORICAL_LOT_SIZE,
        cost_per_lot=COST_PER_LOT,
        current_lot_size=CURRENT_LOT_SIZE
    )
    
    journal.load_data(trades_list)
    
    if journal.data is None or journal.data.empty:
        print("Critical Error: No valid trade data was processed by TradingJournal after loading (e.g., all rows dropped due to missing essential values). Cannot generate reports. Exiting.")
        exit(1)
        
    print(f"Data loaded into journal: {len(journal.data)} trades processed.")
    journal.calculate_metrics()
    
    print("\nMetrics Calculated (Net of Costs):")
    if journal.metrics:
        for key, value in journal.metrics.items():
            print(f"  {key}: {value}")
    else:
        print("  No metrics were calculated.")
    
    print(f"\nGenerating reports in '{REPORTS_OUTPUT_DIRECTORY}' directory...")
    journal.generate_and_open_reports(out_dir=REPORTS_OUTPUT_DIRECTORY)

Successfully loaded 150 raw trade records from 'D:\\TBPL.db'.
Data loaded into journal: 150 trades processed.

Metrics Calculated (Net of Costs):
  Starting Capital: ₹14,00,000.00
  Start Date: 2024-07-16
  End Date: 2026-01-21
  Trading Days: 554
  Total Trades: 150
  Winning Trades: 62
  Losing Trades: 88
  Winning Rate: 41.33%
  Avg Win: ₹45,386.87
  Avg Loss: ₹-,27,614.35
  Win/Loss Ratio: 1.64
  Total Net P&L: ₹3,83,923.00
  Total Net P&L %: 27.42%
  Max Drawdown: ₹-5,94,382.50
  Max Drawdown %: -27.22%
  Max Win Streak: 5
  Max Loss Streak: 10
  Max DD Duration (days): 268
  Current DD Duration (days): 268
  Historical Lot Size: 75
  Cost per Lot: ₹10.00
  Highest Cumulative Profit: ₹7,83,780.00
  Current Capital: ₹17,83,923.00
  Highest Capital Value: ₹21,83,780.00
  Current Lot Size: 6
  Capital Allocation per Lot: ₹4,00,000.00

Generating reports in 'trading_reports_net_final' directory...

Reports and charts generated in 'd:\Trading Journal Tool (Finalised)\trading_reports_ne